# Solve Single Crosswalk using QMDP and SARSOP

In [1]:
using POMDPs, StatsBase, POMDPToolbox, DeepRL, Parameters, GridInterpolations
using AutomotiveDrivingModels,AutoViz
using Reel 
using QMDP, SARSOP

In [2]:
include("AutomotivePOMDPs.jl")
using AutomotivePOMDPs

In [3]:
rng = MersenneTwister(1);

## Initialize Crosswalk environment

In [32]:
pomdp = OCPOMDP(ΔT = 0.1, no_ped_prob=0., p_birth = 0.01)

AutomotivePOMDPs.OCPOMDP(AutomotivePOMDPs.CrosswalkEnv(Roadway, AutomotiveDrivingModels.Lane(LaneTag(2, 1), AutomotiveDrivingModels.CurvePt[CurvePt({25.000, -10.000, 1.571}, 0.000, 0.000, NaN), CurvePt({25.000, 10.000, 1.571}, 20.000, 0.000, NaN)], 6.0, AutomotiveDrivingModels.SpeedLimit(-Inf, Inf), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneConnection[], AutomotiveDrivingModels.LaneConnection[]), AutomotiveDrivingModels.ConvexPolygon[ConvexPolygon: len 4 (max 4 pts)
	VecE2(15.000, -1.500)
	VecE2(15.000, -4.500)
	VecE2(21.500, -4.500)
	VecE2(21.500, -1.500)
], AutomotivePOMDPs.CrosswalkParams(2, 50.0, 3.0, 20.0, 6.0, 5.0, 37.0, 8.0, 100, 0.5, 2.0, 10.0)), AutomotiveDrivingModels.LidarSensor([-1.50796, -1.44513, -1.3823, -1.31947, -1.25664, -1.19381, -1.13097, -1.06814, -1.00531, -0.942478  …  1.00531, 1.06814, 1.13097, 1.19381, 1.25664, 1.31947, 1.3823, 1.44513, 1.50796, 1.5708], [6.938

In [33]:
policy = solve(RandomSolver(rng), pomdp)
up = updater(policy);

In [36]:
hr = HistoryRecorder(rng=rng, max_steps=200)
@time hist = simulate(hr, pomdp, policy, up);

  0.004954 seconds (6.22 k allocations: 2.860 MiB)


In [37]:
duration, fps, render_hist = animate_hist(pomdp, hist, SceneOverlay[])
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpu8M4NH", 0x00000000000000c8, 10.0, nothing)

## Solve using QMDP

In [5]:
solver = QMDPSolver(max_iterations=100, tolerance=1e-3)

QMDP.QMDPSolver(100, 0.001)

In [6]:
qmdp_policy = solve(solver, pomdp, verbose=true)

[Iteration 1   ] residual:          1 | iteration runtime:    510.625 ms, (     0.511 s total)
[Iteration 2   ] residual:       0.95 | iteration runtime:    497.378 ms, (      1.01 s total)
[Iteration 3   ] residual:      0.903 | iteration runtime:    493.015 ms, (       1.5 s total)
[Iteration 4   ] residual:      0.857 | iteration runtime:    494.190 ms, (         2 s total)
[Iteration 5   ] residual:      0.811 | iteration runtime:    493.056 ms, (      2.49 s total)
[Iteration 6   ] residual:      0.762 | iteration runtime:    490.577 ms, (      2.98 s total)
[Iteration 7   ] residual:      0.707 | iteration runtime:    493.424 ms, (      3.47 s total)
[Iteration 8   ] residual:      0.649 | iteration runtime:    493.438 ms, (      3.97 s total)
[Iteration 9   ] residual:      0.539 | iteration runtime:    490.217 ms, (      4.46 s total)
[Iteration 10  ] residual:      0.472 | iteration runtime:    494.516 ms, (      4.95 s total)
[Iteration 11  ] residual:       0.42 | iteration 

QMDP.QMDPPolicy{AutomotivePOMDPs.SingleOCPOMDP,AutomotivePOMDPs.SingleOCAction}([0.445726 0.445726 0.445726 0.449222; 0.478471 0.478471 0.478471 0.479646; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], AutomotivePOMDPs.SingleOCAction[AutomotivePOMDPs.SingleOCAction(-4.0), AutomotivePOMDPs.SingleOCAction(-2.0), AutomotivePOMDPs.SingleOCAction(0.0), AutomotivePOMDPs.SingleOCAction(2.0)], AutomotivePOMDPs.SingleOCPOMDP(AutomotivePOMDPs.CrosswalkEnv(Roadway, AutomotiveDrivingModels.Lane(LaneTag(2, 1), AutomotiveDrivingModels.CurvePt[CurvePt({25.000, -10.000, 1.571}, 0.000, 0.000, NaN), CurvePt({25.000, 10.000, 1.571}, 20.000, 0.000, NaN)], 6.0, AutomotiveDrivingModels.SpeedLimit(-Inf, Inf), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneConnection[], AutomotiveDrivingModels.LaneConnection[]), AutomotiveDrivingModels.ConvexPolygon[ConvexPolygon: len 4 (max 4 pts)
	VecE2(15.000, -1.500)
	VecE2(15.000, -4

In [7]:
up = SingleOCUpdater(pomdp);

In [8]:
hr = HistoryRecorder(rng=rng)
@time hist = simulate(hr, pomdp, qmdp_policy, up);

  6.779805 seconds (81.22 M allocations: 4.300 GiB, 10.39% gc time)


In [9]:
duration, fps, render_hist = animate_hist(pomdp, hist)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmp5QjFW5", 0x0000000000000015, 2.0, nothing)

## QMDP Utility Fusion

In [ ]:
const DecBelief = Dict{Int64, SingleOCDistribution}
const DecState = Dict{Int64, SingleOCState}
const DecObs = Dict{Int64, SingleOCObs}

In [ ]:
pomdp = OCPOMDP()

In [ ]:
up = MixedUpdater(pomdp, pomdp.ΔT)

In [ ]:
hr = HistoryRecorder(rng=rng)
s0 = initial_state(pomdp, rng)
@time hist = simulate(hr, pomdp, qmdp_policy, up, d0, s0);

## Solve using SARSOP

In [10]:
solver = SARSOPSolver(randomization=true, precision = 0.001, timeout = 600.)


SARSOP.SARSOPSolver(Dict{AbstractString,Any}(Pair{AbstractString,Any}("randomization", ""),Pair{AbstractString,Any}("timeout", 3600.0)))

In [11]:
sarsop_policy = solve(solver, pomdp,
               SARSOP.create_policy(solver, pomdp, "crosswalk.policy"), pomdp_file_name="crosswalk.pomdpx" )

Generating a pomdpx file: crosswalk.pomdpx

Loading the model ...
  input file   : crosswalk.pomdpx
  loading time : 45.64s 

SARSOP initializing ...
  initialization time : 90.60s

-------------------------------------------------------------------------------
 Time   |#Trial |#Backup |LBound    |UBound    |Precision  |#Alphas |#Beliefs  
-------------------------------------------------------------------------------
 90.6    0       0        12.1169    12.1191    0.00217179  4        1        
 105.38  4       19       12.1182    12.1191    0.000954115 17       11       
-------------------------------------------------------------------------------

SARSOP finishing ...
  target precision reached
  target precision  : 0.001000
  precision reached : 0.000954 

-------------------------------------------------------------------------------
 Time   |#Trial |#Backup |LBound    |UBound    |Precision  |#Alphas |#Beliefs  
-------------------------------------------------------------------

SARSOP.POMDPPolicy("crosswalk.policy", POMDPXFiles.POMDPAlphas([10.2753 9.80559 … 10.2753 10.2753; 10.3449 9.93102 … 10.3449 10.3449; … ; 19.9998 19.9998 … 19.9999 19.9999; 19.9998 19.9998 … 19.9999 19.9999], [3, 1, 3, 3, 3, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3]), AutomotivePOMDPs.SingleOCPOMDP(AutomotivePOMDPs.CrosswalkEnv(Roadway, AutomotiveDrivingModels.Lane(LaneTag(2, 1), AutomotiveDrivingModels.CurvePt[CurvePt({25.000, -10.000, 1.571}, 0.000, 0.000, NaN), CurvePt({25.000, 10.000, 1.571}, 20.000, 0.000, NaN)], 6.0, AutomotiveDrivingModels.SpeedLimit(-Inf, Inf), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneConnection[], AutomotiveDrivingModels.LaneConnection[]), AutomotiveDrivingModels.ConvexPolygon[ConvexPolygon: len 4 (max 4 pts)
	VecE2(15.000, -1.500)
	VecE2(15.000, -4.500)
	VecE2(21.500, -4.500)
	VecE2(21.500, -1.500)
], AutomotivePOMDPs.CrosswalkParams(2, 50.0, 3.0, 20.0, 6.0, 5.0, 

In [37]:
hr = HistoryRecorder(rng=rng)
@time hist = simulate(hr, pomdp, sarsop_policy, DiscreteUpdater(pomdp));

 11.688374 seconds (165.43 M allocations: 13.435 GiB, 16.18% gc time)


In [38]:
duration, fps, render_hist = animate_hist(pomdp, hist)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpoBsqbv", 0x0000000000000009, 2.0, nothing)